In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Load API Credentials
with open('/Users/default.DESKTOP-0LDO0LD/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
    
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

print("Done!")

Done!


In [2]:
# set our API call parameters. When we want to search somthing else just change these parameters. 
LOCATION = 'NY,NY'
TERM = 'Pasta'

# Directory to save results from api calls
JSON_FILE = "Data/pasta-api.json"

In [3]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists:
        ## Check if user wants to delete if exists
        if delete_if_exists:            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
   
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)


In [4]:
## Create a new empty json file (delete the previous version if it exists)
create_json_file(JSON_FILE, True)

[i] Data/pasta-api.json not found. Saving empty list to new file.


In [5]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)

## How many results total?
total_results = results['total']

## How many did we get the details for?
results_per_page = len(results['businesses'])

# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

- 0 previous results found.


295

In [6]:
for i in tqdm_notebook(range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
        
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/295 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [7]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media3.fl.yelpcdn.com/bphoto/Nn4I1S...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,3678,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,hdiuRS9sVZSMReZm4oV5SA,da-andrea-new-york,Da Andrea,https://s3-media2.fl.yelpcdn.com/bphoto/ZbJxx7...,False,https://www.yelp.com/biz/da-andrea-new-york?ad...,1596,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.736218, 'longitude': -73.99597}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '35 W 13th St', 'address2': '', '...",+12123671979,(212) 367-1979,3424.729337
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2787,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,ePJlqiIhoVKIJW69IE53rg,botte-brooklyn-brooklyn,Botte Brooklyn,https://s3-media1.fl.yelpcdn.com/bphoto/qajQ25...,False,https://www.yelp.com/biz/botte-brooklyn-brookl...,20,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.69446, 'longitude': -73.98316}","[pickup, delivery]",NaN,"{'address1': '327 Gold St', 'address2': None, ...",,,1546.804489
4,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1252,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.718554, 'longitude': -73.997447}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,kxUKKlbm4sK2v6e8xHKB9A,casanova-restaurant-brooklyn,Casanova Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/A_fGiI...,False,https://www.yelp.com/biz/casanova-restaurant-b...,73,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.73413, 'longitude': -73.95206}","[pickup, delivery]",$$,"{'address1': '338 McGuinness Blvd', 'address2'...",+17183890990,(718) 389-0990,4780.928092
996,hFeGYPt9M5dd_KXG9a_ePQ,patsy-s-pizzeria-new-york,Patsy’s Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/FUfbat...,False,https://www.yelp.com/biz/patsy-s-pizzeria-new-...,448,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.76177, 'longitude': -73.96575}","[pickup, delivery]",$$,"{'address1': '206 E 60th St', 'address2': '', ...",+12126889707,(212) 688-9707,6715.014923
997,_J2wfGgCtx4wT05VK3S75w,amore-pizza-cafe-new-york,Amore Pizza Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/yW4HiE...,False,https://www.yelp.com/biz/amore-pizza-cafe-new-...,31,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.76833171568801, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '370 W 58th St', 'address2': '', ...",+12125814200,(212) 581-4200,7035.247930
998,yY15aCwP6lRnEfSlbV_mhA,mama-mals-italian-cuisine-austin-3,Mama Mal's Italian Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/wik1f5...,True,https://www.yelp.com/biz/mama-mals-italian-cui...,42,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 30.253537, 'longitude': -97.763551}",[delivery],$,"{'address1': '1320 S Lamar Blvd', 'address2': ...",+17182087412,(718) 208-7412,2886.874112
999,KgxzEspPDlfZ-6_HX70mfg,ninos-aq-astoria,Ninos Aq,https://s3-media1.fl.yelpcdn.com/bphoto/ScAu7D...,False,https://www.yelp.com/biz/ninos-aq-astoria?adju...,224,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.774944, 'longitude': -73.908585}","[pickup, delivery]",$$,"{'address1': '35-01 Ditmars Blvd', 'address2':...",+17187770010,(718) 777-0010,10576.500192


In [8]:
# final_df.duplicated().sum() # Error: unhashable type: 'list'

# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

## Drop duplicate ids and confirm there are no more duplicates
# final_df = final_df.drop_duplicates(subset='id')
# final_df.duplicated(subset='id').sum()

0

In [9]:
final_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media3.fl.yelpcdn.com/bphoto/Nn4I1S...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,3678,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,hdiuRS9sVZSMReZm4oV5SA,da-andrea-new-york,Da Andrea,https://s3-media2.fl.yelpcdn.com/bphoto/ZbJxx7...,False,https://www.yelp.com/biz/da-andrea-new-york?ad...,1596,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.736218, 'longitude': -73.99597}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '35 W 13th St', 'address2': '', '...",+12123671979,(212) 367-1979,3424.729337
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2787,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,ePJlqiIhoVKIJW69IE53rg,botte-brooklyn-brooklyn,Botte Brooklyn,https://s3-media1.fl.yelpcdn.com/bphoto/qajQ25...,False,https://www.yelp.com/biz/botte-brooklyn-brookl...,20,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.69446, 'longitude': -73.98316}","[pickup, delivery]",NaN,"{'address1': '327 Gold St', 'address2': None, ...",,,1546.804489
4,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1252,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.718554, 'longitude': -73.997447}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,kxUKKlbm4sK2v6e8xHKB9A,casanova-restaurant-brooklyn,Casanova Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/A_fGiI...,False,https://www.yelp.com/biz/casanova-restaurant-b...,73,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.73413, 'longitude': -73.95206}","[pickup, delivery]",$$,"{'address1': '338 McGuinness Blvd', 'address2'...",+17183890990,(718) 389-0990,4780.928092
996,hFeGYPt9M5dd_KXG9a_ePQ,patsy-s-pizzeria-new-york,Patsy’s Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/FUfbat...,False,https://www.yelp.com/biz/patsy-s-pizzeria-new-...,448,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.76177, 'longitude': -73.96575}","[pickup, delivery]",$$,"{'address1': '206 E 60th St', 'address2': '', ...",+12126889707,(212) 688-9707,6715.014923
997,_J2wfGgCtx4wT05VK3S75w,amore-pizza-cafe-new-york,Amore Pizza Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/yW4HiE...,False,https://www.yelp.com/biz/amore-pizza-cafe-new-...,31,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.76833171568801, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '370 W 58th St', 'address2': '', ...",+12125814200,(212) 581-4200,7035.247930
998,yY15aCwP6lRnEfSlbV_mhA,mama-mals-italian-cuisine-austin-3,Mama Mal's Italian Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/wik1f5...,True,https://www.yelp.com/biz/mama-mals-italian-cui...,42,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 30.253537, 'longitude': -97.763551}",[delivery],$,"{'address1': '1320 S Lamar Blvd', 'address2': ...",+17182087412,(718) 208-7412,2886.874112


In [10]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pasta.csv.gz', compression='gzip',index=False)